In [12]:
import pandas as pd
import numpy as np

In [13]:
pop = pd.read_csv('fr_population.csv')
animaux = pd.read_csv('fr_animaux.csv')
vege = pd.read_csv('fr_vegetaux.csv')
cereales = pd.read_csv ('fr_céréales.csv')
s_alim = pd.read_csv('fr_sousalimentation.csv')

## Création des différentes tables demandées pour l'import dans la base de donnée SQL

In [14]:
population = pop[['Zone', 'Code zone', 'Année','Valeur']]
population = population.rename(columns={'Zone' : 'pays', 
                                                  'Code zone' : 'code_pays',
                                                  'Année' : 'annee',
                                                  'Valeur' : 'population'})
# Suppression des données en double si il y en a
population = population.drop_duplicates()
# Création d'un index afin de récupérer la ligne concernée
indexNames = population[population['pays'] == 'Chine, continentale' ].index
# Suppression de la ligne "Chine" du dataframe
population.drop(indexNames , inplace=True)
population.sort_values(by = 'population', ascending=False)

,pays,code_pays,annee,population
33,Chine,351,2013,1416667
75,Inde,100,2013,1252140
55,États-Unis d'Amérique,231,2013,320051
76,Indonésie,101,2013,249866
24,Brésil,21,2013,200362
...,...,...,...,...
89,Kiribati,83,2013,102
6,Antigua-et-Barbuda,8,2013,90
47,Dominique,55,2013,72
20,Bermudes,17,2013,65


In [15]:
population.to_csv("population.csv", index = False)

In [17]:
# Création d'un index afin de récupérer la ligne concernée
indexNames2 = animaux[ animaux['Zone'] == 'Chine, continentale' ].index
# Supprimer la ligne Chine du dataframe
animaux.drop(indexNames2 , inplace=True)

In [18]:
# Création d'un index afin de récupérer la ligne concernée
indexNames3 = vege[vege['Zone'] == 'Chine, continentale' ].index
# Supprimer la ligne Chine du dataframe
vege.drop(indexNames3 , inplace=True)

In [19]:
# Création d'un index afin de récupérer la ligne concernée
indexNames4 = cereales[cereales['Zone'] == 'Chine, continentale' ].index
# Supprimer la ligne Chine continentale du dataframe
cereales.drop(indexNames4 , inplace=True)

In [20]:
# Création d'un index afin de récupérer la ligne concernée
indexNames5 = s_alim[s_alim['Zone'] == 'Chine, continentale' ].index
# Supprimer la ligne Chine du dataframe
s_alim.drop(indexNames5 , inplace=True)

In [21]:
bilan_alim_anim = animaux
bilan_alim_anim['origin'] = 'animal'
bilan_alim_veg = vege
bilan_alim_veg['origin'] = 'vegetal'
dispo_alim = bilan_alim_anim.append(bilan_alim_veg)
del bilan_alim_anim
del bilan_alim_veg
dispo_alim = pd.pivot_table(dispo_alim, index=['Code zone', 'Zone', 'Code Produit', 'Produit', 'origin'], columns = 'Élément', values = 'Valeur', aggfunc=np.sum).reset_index()
dispo_alim.columns.name = None
dispo_alim = dispo_alim.rename(columns = {'Zone' : 'pays', 
                                          'Code zone' : 'code_pays',
                                          'Année' : 'annee',
                                          'Produit' : 'produit', 
                                          'Code Produit' : 'code_produit', 
                                          'Disponibilité alimentaire en quantité (kg/personne/an)' : 'dispo_kg_p',
                                          'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
                                          'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr',
                                          'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',})
dispo_alim = pd.merge(dispo_alim, population, on =['pays', 'code_pays'])
dispo_alim['dispo_alim_tonnes'] = (dispo_alim['dispo_kg_p']/ 1000) * (dispo_alim['population']*1000)
dispo_alim = dispo_alim[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'origin', 'dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr']]
dispo_alim.to_csv("dispo_alim.csv", index = False)
dispo_alim

,pays,code_pays,annee,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,Arménie,1,2013,Blé,2511,vegetal,3.887962e+05,1024.0,30.52,3.60
1,Arménie,1,2013,Orge,2513,vegetal,0.000000e+00,0.0,0.00,0.00
2,Arménie,1,2013,Maïs,2514,vegetal,8.931000e+01,0.0,0.01,NaN
3,Arménie,1,2013,Seigle,2515,vegetal,3.572400e+02,1.0,0.02,0.00
4,Arménie,1,2013,Avoine,2516,vegetal,1.101490e+03,2.0,0.09,0.03
...,...,...,...,...,...,...,...,...,...,...
15579,Chine,351,2013,"Alcool, non Comestible",2659,vegetal,NaN,NaN,NaN,NaN
15580,Chine,351,2013,Aliments pour enfants,2680,vegetal,1.275000e+05,1.0,0.04,0.01
15581,Chine,351,2013,Miel,2745,vegetal,3.541668e+05,2.0,0.00,NaN
15582,Chine,351,2013,Riz (Eq Blanchi),2805,vegetal,1.097209e+08,797.0,14.47,2.68


In [22]:
bilan_alim_anim = animaux
bilan_alim_veg = vege
equilibre_prod = bilan_alim_anim.append(bilan_alim_veg)
del bilan_alim_anim
del bilan_alim_veg
equilibre_prod = pd.pivot_table(equilibre_prod, index=['Code zone', 'Zone', 'Année', 'Code Produit', 'Produit'], columns = 'Élément', values = 'Valeur', aggfunc=np.sum).reset_index()
equilibre_prod.columns.name = None
equilibre_prod = equilibre_prod.rename(columns = {'Zone' : 'pays', 
                                          'Code zone' : 'code_pays',
                                          'Année' : 'annee',
                                          'Produit' : 'produit', 
                                          'Code Produit' : 'code_produit',
                                          'Disponibilité intérieure': 'dispo_int',
                                          'Aliments pour animaux' : 'alim_ani',
                                          'Semences' : 'semences',
                                          'Pertes' : 'pertes',
                                          'Traitement' : 'transformes',
                                          'Nourriture' : 'nourriture',
                                          'Autres utilisations (non alimentaire)' : 'autres_utilisations'})
equilibre_prod = equilibre_prod[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_int', 'alim_ani', 'semences', 'pertes', 'transformes', 'nourriture', 'autres_utilisations']]
equilibre_prod.to_csv("equilibre_prod.csv", index = False)
equilibre_prod

,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,transformes,nourriture,autres_utilisations
0,Arménie,1,2013,Blé,2511,554.0,93.0,30.0,32.0,10.0,389.0,0.0
1,Arménie,1,2013,Orge,2513,198.0,137.0,14.0,15.0,7.0,0.0,26.0
2,Arménie,1,2013,Maïs,2514,102.0,96.0,0.0,7.0,NaN,0.0,NaN
3,Arménie,1,2013,Seigle,2515,1.0,1.0,0.0,0.0,NaN,0.0,NaN
4,Arménie,1,2013,Avoine,2516,6.0,4.0,0.0,0.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15579,Chine,351,2013,"Alcool, non Comestible",2659,239.0,NaN,NaN,NaN,NaN,NaN,239.0
15580,Chine,351,2013,Aliments pour enfants,2680,133.0,NaN,NaN,0.0,NaN,132.0,NaN
15581,Chine,351,2013,Miel,2745,350.0,NaN,NaN,NaN,NaN,350.0,0.0
15582,Chine,351,2013,Riz (Eq Blanchi),2805,135024.0,12117.0,4679.0,6406.0,12.0,109725.0,2085.0


In [23]:
sous_nutrition = s_alim[['Zone', 'Code zone', 'Année', 'Valeur']]
sous_nutrition = sous_nutrition.rename(columns = {'Zone':'pays',
                                                  'Code zone' : 'code_pays', 
                                                  'Année' : 'annee',
                                                  'Valeur' : 'nb_personnes'})
sous_nutrition = sous_nutrition.replace('<0.1', 0)
sous_nutrition = sous_nutrition.dropna()
sous_nutrition.to_csv("sous_nutrition.csv", index = False)
sous_nutrition

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2012-2014,7.9
1,Afghanistan,2,2013-2015,8.8
2,Afghanistan,2,2014-2016,9.6
3,Afghanistan,2,2015-2017,10.2
4,Afghanistan,2,2016-2018,10.6
...,...,...,...,...
1015,Zimbabwe,181,2012-2014,6.6
1016,Zimbabwe,181,2013-2015,7.2
1017,Zimbabwe,181,2014-2016,7.8
1018,Zimbabwe,181,2015-2017,8.2


## Les tables sont ensuite importées dans SQLite Studio dans une base de donnée appelée "FAO"